In [2]:
import pandas as pd
from nltk import word_tokenize
import numpy as np

In [3]:
df = pd.read_csv('train.csv')
df['words'] = df.text.apply(lambda text: word_tokenize(text.lower()))
dfw = df.explode('words')[['id', 'author', 'words']]
dfw

,id,author,words
0,id26305,EAP,this
0,id26305,EAP,process
0,id26305,EAP,","
0,id26305,EAP,however
0,id26305,EAP,","
...,...,...,...
19578,id00393,HPL,altogether
19578,id00393,HPL,that
19578,id00393,HPL,of
19578,id00393,HPL,mirth


In [4]:
tokens = []
token_doc_index = []
unique_words_dict = {}
unique_words = []
for index, row in dfw.iterrows():
    if len(tokens) % 20000 == 0:
        print(len(tokens), 'of', len(dfw))
    token = row['words']
    if not token in unique_words_dict:
        unique_words_dict[token] = len(unique_words)
        unique_words.append(token)
    tokens.append(unique_words_dict[token])
    token_doc_index.append(index)
tokens = np.array(tokens)
token_doc_index = np.array(token_doc_index)
unique_words = np.array(unique_words)

0 of 594922
20000 of 594922
40000 of 594922
60000 of 594922
80000 of 594922
100000 of 594922
120000 of 594922
140000 of 594922
160000 of 594922
180000 of 594922
200000 of 594922
220000 of 594922
240000 of 594922
260000 of 594922
280000 of 594922
300000 of 594922
320000 of 594922
340000 of 594922
360000 of 594922
380000 of 594922
400000 of 594922
420000 of 594922
440000 of 594922
460000 of 594922
480000 of 594922
500000 of 594922
520000 of 594922
540000 of 594922
560000 of 594922
580000 of 594922


In [5]:
k = 10
alpha =  min(0.1,50/k)
beta = 0.01

In [6]:
Z = np.random.choice(np.arange(k), len(tokens), [1/k]*k)
cdk = np.array([[0 for t in range(k)] for d in range(len(df))])                                      
ckv = np.array([[0 for t in range(k)] for v in range(len(unique_words))])
cd = np.zeros(len(df))
ck = np.zeros(k)

In [7]:
for i in range(len(Z)):
    cdk[token_doc_index[i], Z[i]] += 1
    ckv[tokens[i], Z[i]] += 1
    cd[token_doc_index[i]] += 1
    ck[Z[i]] += 1

In [8]:
def gen_p(i):
    ps = []
    for t in range(k):
        num1 = alpha + cdk[token_doc_index[i], t]
        denom1 = k*alpha + cd[token_doc_index[i]]
        num2 = beta + ckv[tokens[i], t]
        denom2 = len(unique_words)*beta + ck[t]
        p = num1/denom1*num2/denom2
        ps.append(p)
    return ps

for it in range(3):
    for i in range(len(tokens)):
        if i % 3000 == 0:
            print('iteration:', it, 'in, token:', i, 'of', len(tokens))
        cdk[token_doc_index[i], Z[i]] -=1
        ckv[tokens[i], Z[i]] -= 1
        cd[token_doc_index[i]] -= 1
        ck[Z[i]] -= 1
        p = np.array(gen_p(i))
        p = p/p.sum()
        new_topic = np.random.choice(np.arange(k), p=p)
        Z[i] = new_topic
        cdk[token_doc_index[i], new_topic] +=1
        ckv[tokens[i], new_topic] += 1
        cd[token_doc_index[i]] += 1
        ck[new_topic] += 1
        

iteration: 0 in, token: 0 of 594922
iteration: 0 in, token: 3000 of 594922
iteration: 0 in, token: 6000 of 594922
iteration: 0 in, token: 9000 of 594922
iteration: 0 in, token: 12000 of 594922
iteration: 0 in, token: 15000 of 594922
iteration: 0 in, token: 18000 of 594922
iteration: 0 in, token: 21000 of 594922
iteration: 0 in, token: 24000 of 594922
iteration: 0 in, token: 27000 of 594922
iteration: 0 in, token: 30000 of 594922
iteration: 0 in, token: 33000 of 594922
iteration: 0 in, token: 36000 of 594922
iteration: 0 in, token: 39000 of 594922
iteration: 0 in, token: 42000 of 594922
iteration: 0 in, token: 45000 of 594922
iteration: 0 in, token: 48000 of 594922
iteration: 0 in, token: 51000 of 594922
iteration: 0 in, token: 54000 of 594922
iteration: 0 in, token: 57000 of 594922
iteration: 0 in, token: 60000 of 594922
iteration: 0 in, token: 63000 of 594922
iteration: 0 in, token: 66000 of 594922
iteration: 0 in, token: 69000 of 594922
iteration: 0 in, token: 72000 of 594922
iterati

In [9]:
cdk_final = []
for i in range(len(cdk)):
    cdk_final.append(cdk[i]/cd[i])
cdk_final = np.array(cdk_final)
cdk_final

array([[0.16666667, 0.16666667, 0.        , ..., 0.        , 0.02083333,
        0.20833333],
       [0.33333333, 0.4       , 0.06666667, ..., 0.13333333, 0.        ,
        0.06666667],
       [0.12195122, 0.        , 0.        , ..., 0.19512195, 0.        ,
        0.14634146],
       ...,
       [0.        , 0.        , 0.        , ..., 0.26666667, 0.26666667,
        0.        ],
       [0.38888889, 0.11111111, 0.05555556, ..., 0.        , 0.16666667,
        0.        ],
       [0.16666667, 0.        , 0.        , ..., 0.        , 0.        ,
        0.125     ]])

In [10]:
ckv_final = []
for i in range(len(unique_words)):
    ckv_final.append(ckv[i]/np.sum(ckv[i]))
ckv_final = np.array(ckv_final)
ckv_final

array([[0.10830463, 0.08725603, 0.1010333 , ..., 0.0822809 , 0.10907003,
        0.1010333 ],
       [0.47058824, 0.        , 0.        , ..., 0.        , 0.23529412,
        0.11764706],
       [0.09345892, 0.1032967 , 0.10023548, ..., 0.10978545, 0.09945055,
        0.10355835],
       ...,
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])